In [ ]:
import numpy as np
import torch as tc
import os,math,glob,tqdm,ujson,librosa,random,time
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt

class my_dic:
    def __init__(self) -> None:
        self.data=[]
        self.dict={}
    
    def load_data(self):
        self.dict=dict(zip(self.data,range(len(self.data))))
        self.undict=dict(zip(range(len(self.data)),self.data))

    def ck(self,char):
        if char not in self.data:
            self.data.append(char)
    
    def transform(self,str:list) ->tc.Tensor:
        if len(self.dict)!=len(self.data):
            self.load_data()
        
        tenser=tc.zeros(len(str),len(self.dict))
        for x in range(tenser.shape[0]):
            str[x]
            tenser[x,self.dict[str[x]]]=1
        return tenser
    
    def untransform(self,ts:tc.Tensor):
        data=[]
        tc=ts==1
        data=np.array(self.data)
        data=data[tc]
        return data

    def save(self,filepath):
        with open(filepath+"dc","w")as file:
            ujson.dump(self.data,file)
    
    def load(self,filepath):
        with open(filepath+"dc","r")as file:
            self.data=ujson.load(file)
            self.load_data()     
   
class Module(nn.Module):
    def __init__(self):
        super().__init__()
        self.optimizer_list=[]
    
    def init(self,pack:list,optimize=tc.optim.AdamW):
        self.Model_pack=pack
        self.optimizer_list=[optimize(x.parameters()) for x in pack]
    
    def load_checkpoint(self,checkpoint):
        if not os.path.exists(checkpoint):
            print("load error checkpoint")
            return None
        if checkpoint != 'No':
            print("loading checkpoint...")
            model_dict = self.state_dict()
            modelCheckpoint = tc.load(checkpoint)
            pretrained_dict = modelCheckpoint['state_dict']
            try:
                for x in range(len(self.optimizer_list)):
                    self.optimizer_list[x].load_state_dict(modelCheckpoint['optimizer'][x])
            except:
                print('not loaded optimizer')
            # 过滤操作
            new_dict={}
            for k,v in pretrained_dict.items():
                if k in model_dict.keys() and model_dict[k].shape==v.shape:
                    v=v.to(model_dict[k].device)
                    new_dict[k]=v
                else :
                    try:
                        print(f"pass: {k} data_v: {v.shape} model_v: {model_dict[k].shape}")
                    except:
                        print(f"pass: {k} data_v: {v.shape} model_v: None")
                    input("enter?")
                    v:tc.Tensor
                    #cache=v.clone()
                    if k not in model_dict.keys():
                        print("key error")
                        continue
                    model_dict[k]*=0
                    cache=model_dict[k].shape
                    try:
                        v=v.to(model_dict[k].device)
                        cache=v.shape
                        if v.ndim==4:
                            model_dict[k][:cache[0],:cache[1],:cache[2],:cache[3]]+=v
                        elif v.ndim==3:
                            model_dict[k][:cache[0],:cache[1],:cache[2]]+=v
                        elif v.ndim==2:
                            model_dict[k][:cache[0],:cache[1]]+=v
                        elif v.ndim==1:
                            model_dict[k][:cache[0]]+=v
                        else:
                            break
                        print("load finish")
                    except:
                        #v=cache
                        print("load error")
                    
            #new_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys()}
            model_dict.update(new_dict)
            # 打印出来，更新了多少的参数
            print('Total : {}, update: {}'.format(len(pretrained_dict), len(new_dict)))
            self.load_state_dict(model_dict)
            print("loaded finished!")
            # 如果不需要更新优化器那么设置为false
            
        else:
            print('No checkpoint is included')
        print("load checkpoint Succeeded")
        self.save()

    def save(self,path=None,loss=None):
        try:
            cache=self.save_x
        except:
            self.save_x=0 
        if len(self.optimizer_list):
            optimizer_data=[x.state_dict() for x in self.optimizer_list]
        else :
            optimizer_data=[]
        if path:
            tc.save({"state_dict":self.state_dict(),"optimizer":optimizer_data},path+"_%2f.pth"%loss)
        else:
            if "model_data" not in glob.glob("*"):
                os.system("mkdir model_data")
            if loss==None:
                tc.save({"data":self.save_x,"state_dict":self.state_dict(),
                    "optimizer":optimizer_data
                         },"model_data/"+str(self.save_x)+".pth")
            else :
                tc.save({"data":self.save_x,"state_dict":self.state_dict(),
                        "optimizer":optimizer_data
                         },"model_data/"+str(self.save_x)+"_%2f.pth"%loss)
                
            self.save_x+=1

def preprocess_voice(voice_path):
    audio, _ = librosa.core.load(voice_path, sr=None, mono=True)

    melspec = librosa.feature.melspectrogram(
        audio,
        sr=32_000,
        n_fft=800,
        hop_length=320,
        n_mels=128,
        fmin=20,
        fmax=14_000,
        power=1,
    )
    x_train=librosa.pcen(
        melspec * (2 ** 31),
        time_constant=0.06,
        eps=1e-6,
        gain=0.8,
        power=0.25,
        bias=10,
        sr=32_000,
        hop_length=320,
    )
    return tc.from_numpy(x_train.T)

##############################################################################################################
def conv_bn_relu(in_channels:int,out_channels:int,kernel_size,stride=1,padding=0,groups:int=1,relu=True,conv=nn.Conv2d):
    if relu:
        return nn.Sequential(conv(in_channels,out_channels,kernel_size,stride=stride,padding=padding,groups=groups,),
                            nn.BatchNorm2d(out_channels,affine=True),
                            nn.LeakyReLU(1e-4))
    else: 
        return nn.Sequential(conv(in_channels,out_channels,kernel_size,stride=stride,padding=padding,groups=groups,),
                            nn.BatchNorm2d(out_channels,affine=True))

def Linear_dp_relu(input_size,output_size,dropout_rate=0,relu=True):
    cache=[nn.Linear(input_size,output_size)]
    if relu:
        cache.append(nn.LeakyReLU(1e-4))
    if dropout_rate:
        cache.append(nn.Dropout(dropout_rate))
    return nn.Sequential(*cache)

class Bottleneck1(nn.Module):#Bottleneck1
    def __init__(self,in_channels:int,out_channels:int,kernel_size,stride=1,padding=1,groups:int=1,s=False):
        super().__init__()
        self.cnn1=conv_bn_relu(in_channels,in_channels,1,stride,0,groups)
        self.cnn2=conv_bn_relu(in_channels,in_channels,kernel_size,stride,padding,groups)
        self.cnn3=conv_bn_relu(in_channels,out_channels,1,stride,0,groups,relu=False)
        self.cnn4=conv_bn_relu(in_channels,out_channels,1,stride,0,groups,relu=False)
        #self.a=tc.nn.Parameter(-tc.ones(1))
        self.s=s
    def forward(self,x:tc.Tensor):
        x1=self.cnn1.forward(x)
        x2=self.cnn4.forward(x1)
        if self.s:
            x1=tc.max_pool2d(x1,self.s)
            x2=tc.max_pool2d(x2,self.s)
        x1=self.cnn2.forward(x1)
        x1=self.cnn3.forward(x1)
        
        #a=tc.sigmoid(self.a)
        x=x1+x2
        #x=x1*a+x2*(1-a)
        return F.leaky_relu(x,1e-4)

class Bottleneck2(nn.Module):#Bottleneck2
    def __init__(self,in_channels:int,hid_channels:int,kernel_size,stride=1,padding=1,groups:int=1,s=False):
        super().__init__()
        self.cnn1=conv_bn_relu(in_channels,hid_channels,1,stride,0,groups)
        self.cnn2=conv_bn_relu(hid_channels,hid_channels,kernel_size,stride,padding,groups)
        self.cnn3=conv_bn_relu(hid_channels,in_channels,1,stride,0,groups,relu=False)
        #self.a=tc.nn.Parameter(-tc.ones(1))
        self.s=s
    def forward(self,x:tc.Tensor):
        x1=self.cnn1.forward(x)
        x1=self.cnn2.forward(x1)
        x1=self.cnn3.forward(x1)
        
        #a=tc.sigmoid(self.a)
        x=x1+x
        #x=x1*a+x*(1-a)
        return F.leaky_relu(x,1e-4)
        
class Generate(nn.Module):
    def __init__(self,dropout_rate=0):
        super().__init__()
        #input=(batch,152+1024=1176)
        #outpu=(batch,500,128)
        self.dp=nn.Dropout(dropout_rate)
                
        class D_Linear(nn.Module):
            def __init__(self,input_size:tuple=(64,64),output_size:tuple=(128,128),dropout=True):
                super().__init__()
                self.size=output_size
                self.dnn1=Linear_dp_relu(input_size[0],output_size[0],dropout_rate=dropout_rate*dropout)
                self.dnn2=Linear_dp_relu(input_size[1],output_size[1],dropout_rate=dropout_rate*dropout)
            def forward(self,x:tc.Tensor):
                #(batch,c,h,w)
                shape=x.shape
                x=x.reshape(shape[0]*shape[1],shape[2],shape[3])
                x=self.dnn1.forward(x)
                #(batch,c,h,w1)
                x=x.transpose(-1,-2)
                #(batch,c,w1,h)
                x=self.dnn2.forward(x)
                #(batch,c,w1,h1)
                x=x.reshape(shape[0],shape[1],self.size[1],self.size[0])
                return x
            
        self.dnn1=Linear_dp_relu(1176,2048,dropout_rate=dropout_rate)
        self.dnn2=Linear_dp_relu(32,64,dropout_rate=dropout_rate)
        self.cnn1=nn.Sequential(Bottleneck1(1,16,3),Bottleneck2(16,8,3))
        self.dnn3=D_Linear((64,64),(128,128))
        self.cnn2=nn.Sequential(Bottleneck1(16,64,3),Bottleneck2(64,16,3),Bottleneck2(64,16,3))
        self.dnn4=D_Linear((128,128),(256,128))
        self.cnn3=nn.Sequential(Bottleneck2(64,16,3),Bottleneck2(64,16,3),Bottleneck1(64,16,3))#shape=(batch,64,256,128)
        self.dnn5=D_Linear((256,128),(500,120),False)
        self.cnn4=nn.Conv2d(16,1,1)
        #self.dnn_test=D_Linear((64,64),(500,120),False)
        
    def forward(self,x:tc.Tensor):
        #input=(batch,152)
        x=tc.cat((x,tc.ones(x.shape[0],1024,device=x.device)),dim=1)
        #x=tc.cat((x,tc.randn(x.shape[0],1024,device=x.device)),dim=1)
        x=self.dnn1.forward(x)
        x=x.reshape(x.shape[0],1,64,32)
        x=self.dnn2.forward(x)#(batch,1,64,64)
        #x=self.dnn_test.forward(x)[:,0].transpose(-1,-2)
        #return tc.sigmoid(x)
        x=self.cnn1.forward(x)#(batch,16,64,64)
        x=self.dnn3.forward(x)#(batch,16,128,128)
        x=self.cnn2.forward(x)#(batch,64,128,128)
        x=self.dnn4.forward(x)#(batch,64,256,128)
        x=self.cnn3.forward(x)#(batch,64,256,128)
        x=self.dnn5.forward(x)#(batch,64,500,120)
        x=self.cnn4.forward(x)[:,0].transpose(-1,-2)#(batch,1,500,120)
        max_=x.max(dim=2)[0].max(dim=1)[0][:,None,None]
        min_=x.min(dim=2)[0].min(dim=1)[0][:,None,None]
        return tc.sigmoid(x)
        #return (min_-x)/(min_-max_)
        
class Model_p(nn.Module):
    def __init__(self,dropout_rate=0):
        super().__init__()
        #input=(batch,500,128)
        number=[1,64,256,512,1024,2048]#cnn_number

        self.cnn1=conv_bn_relu(number[0],number[1],7,stride=1,padding=5)
        self.cnn2=conv_bn_relu(number[1],number[1],3,stride=1,padding=2)
        #torch.Size([10, 64, 128, 64])
        stage1=[Bottleneck1(number[1],number[2],kernel_size=3),
                Bottleneck2(number[2],int(number[2]/4),kernel_size=3),
                Bottleneck2(number[2],int(number[2]/4),kernel_size=3,)]
        self.stage1=nn.Sequential(*stage1)
        #torch.Size([10, 256, 128, 64])
        stage2=[Bottleneck1(number[2],number[3],kernel_size=3,s=2),
                Bottleneck2(number[3],int(number[3]/4),kernel_size=3),
                Bottleneck2(number[3],int(number[3]/4),kernel_size=3),
                Bottleneck2(number[3],int(number[3]/4),kernel_size=3)]
        self.stage2=nn.Sequential(*stage2)
        #torch.Size([10, 512, 64, 32])
        stage3=[Bottleneck1(number[3],number[4],kernel_size=3,s=2),
                Bottleneck2(number[4],int(number[4]/4),kernel_size=3),
                Bottleneck2(number[4],int(number[4]/4),kernel_size=3),
                Bottleneck2(number[4],int(number[4]/4),kernel_size=3),
                Bottleneck2(number[4],int(number[4]/4),kernel_size=3),
                Bottleneck2(number[4],int(number[4]/4),kernel_size=3)]
        self.stage3=nn.Sequential(*stage3)
        #torch.Size([10, 1024, 16, 16])
        stage4=[Bottleneck1(number[4],number[5],kernel_size=3,s=2),
                Bottleneck2(number[5],int(number[5]/4),kernel_size=3),
                Bottleneck2(number[5],int(number[5]/4),kernel_size=3)]
        self.stage4=nn.Sequential(*stage4)
        #torch.Size([10, 2048, 8, 8])
        #torch.Size([10,64,2048])
        class dnn(nn.Module):
            def __init__(self):
                super().__init__()
                self.dnn1=Linear_dp_relu(2048,512,dropout_rate=dropout_rate)
                #torch.Size([10,64,512])
                self.dnn2=Linear_dp_relu(512,128,dropout_rate=dropout_rate)
                #torch.Size([10,64,128])
                #torch.Size([10,2,4096])
                self.dnn3=Linear_dp_relu(4096,2048,dropout_rate=dropout_rate)
                #torch.Size([10,2,2048])
                #torch.Size([10,4096])
                self.dnn4=Linear_dp_relu(4096,1024,dropout_rate=dropout_rate)
                #torch.Size([10,1024])
            
            def forward(self,x:tc.Tensor):
                #torch.Size([10,64,2048])
                x=self.dnn1(x)
                #torch.Size([10,64,512])
                x=self.dnn2(x)
                #torch.Size([10,64,128])
                x=x.reshape(x.shape[0],2,-1)
                #torch.Size([10,2,4096])
                x=self.dnn3(x)
                #torch.Size([10,2,2048])
                x=x.reshape(x.shape[0],-1)
                #torch.Size([10,4096])
                x=self.dnn4(x)
                #torch.Size([10,1024])
                return x

        dnn_set=[dnn() for x in range(4)]
        self.dnn_set=nn.ModuleList(dnn_set)
        
        dnn_output1=Linear_dp_relu(4096,1024,dropout_rate=dropout_rate)
        dnn_output2=Linear_dp_relu(1024,152,dropout_rate=0,relu=False)
        self.dnn_final=nn.Sequential(dnn_output1,dnn_output2)
        #self.dnn_output=self.dnn_final
        class discriminate(nn.Module):
            def __init__(self):
                super().__init__()
                self.dnn1=Linear_dp_relu(4096,1024,dropout_rate=dropout_rate)
                self.dnn2=Linear_dp_relu(1024,128,dropout_rate=dropout_rate)
                self.dnn3=Linear_dp_relu(128,1,dropout_rate=0,relu=False)
            def forward(self,x:tc.Tensor):
                #x.shape=(batch,4096)
                x=self.dnn1.forward(x)
                x=self.dnn2.forward(x)
                x=self.dnn3.forward(x)
                return x
            
        self.discriminate=discriminate()
        
    def referee_data(self):
        return self.discriminate.parameters()
    
    def feature_data(self):
        for x in [self.cnn1.parameters(),self.cnn2.parameters(),
                  self.stage1.parameters(),self.stage2.parameters(),
                  self.stage3.parameters(),self.stage4.parameters(),
                  self.dnn_set.parameters()]:
            for y in x:
                yield y
    
    def feature_forward(self,x:tc.Tensor)->tc.Tensor:
        x=x[:,None]
        
        x=self.cnn1(x)
        x=tc.max_pool2d(x,(4,2),padding=(2,0))
        
        x=self.cnn2(x)
        x=tc.max_pool2d(x,(2,1))

        x=self.stage1.forward(x)
        x=self.stage2.forward(x)
        x=self.stage3.forward(x)
        x=self.stage4.forward(x)
        
        x=x.reshape(x.shape[0],2048,64).transpose(1,2)
        cache=[]
        for i in range(len(self.dnn_set)):
            #x.shape=[batch,64,2048]
            cache.append(self.dnn_set[i].forward(x))
            #output.shape=[batch,1024]
        x=tc.cat(cache,dim=-1)
        #x.shape=[batch,4096]
        return x
    
    def forward(self,x:tc.Tensor,discriminate=False,feature_no_grad=False):
        """if discriminate false ->feature_no_grad=falses"""
        if discriminate:
            if feature_no_grad:
                with tc.no_grad():
                    x=self.feature_forward(x)
            else :
                x=self.feature_forward(x)
            output=self.dnn_final.forward(x)
            d=self.discriminate(x)
            return tc.sigmoid(tc.cat((output,d),dim=1))
            #output.shape=[batch,152+1]
        else :
            x=self.feature_forward(x)
            x=self.dnn_final.forward(x)
            return tc.sigmoid(x)
            #output.shape=[batch,152]

class Model(Module):
    def __init__(self,dropout_rate=0.1,
                 optimizer=tc.optim.AdamW,lr=1e-4,betas=(0.9,0.9),weight_decay=0.1,
                 loss_fc:nn.BCELoss=nn.BCELoss(reduction="sum"),
                 generate:Generate=Generate(0),generate_rate=0.2):
        super().__init__()
        
        self.cnn=Model_p(dropout_rate)
        cache=dict(lr=lr,betas=betas,weight_decay=weight_decay)
        self.loss_fc=loss_fc
        
        #self.generate=generate
        #self.generate_rate=generate_rate
        #self.generate_optimizer=optimizer(self.generate.parameters(),**cache)
        self.optim=optimizer
        self.optim_cache=cache
        self.init_optimizer()
        
        #self.optimizer_list=[self.generate_optimizer,self.cnn_optimizer]
    
    def init_optimizer(self):
        self.cnn_optimizer=self.optim(self.cnn.parameters(),**self.optim_cache)
        self.optimizer_list=[self.cnn_optimizer]

    def forward(self,x:tc.Tensor,discriminate=False):
        #index is record the park of batch 
        #index.shape=(batch,n)
        #x.shape=(batch,2000,257)
        #model output shape=(batch*4,157)
        #use index to reshape to (batch,157)
        #output=tc.nan_to_num(output,0.0)
        #output=(output-output.min())/(output.max()-output.min())
        cache=x.reshape(x.shape[0],4,500,-1)
        max_=cache.max(dim=3)[0].max(dim=2)[0][:,:,None,None]
        min_=cache.min(dim=3)[0].min(dim=2)[0][:,:,None,None]
        cache=(min_-cache)/(min_-max_)
        output=self.cnn(cache.reshape(x.shape[0]*4,500,-1),discriminate)
        del cache
        output=output.reshape(x.shape[0],4,output.shape[-1])
        
        output=output.max(dim=1)[0]
        return output
    
    def train_cnn(self,x:tc.Tensor,y:tc.Tensor,mask=None):
        self.cnn_optimizer.zero_grad()

        if x.shape[1]==500:
            output=self.cnn(x)
        else :
            output=self.forward(x)
        if not (mask is None):
            output=output[:,mask]
            
        loss=self.loss_fc.forward(output,y)/output.shape[0]
        loss.backward()
        self.cnn_optimizer.step()
        return {"output":output,"loss":tc.tensor([loss])}
    
    def train_all(self,x:tc.Tensor,y:tc.Tensor,mask=None):
        #https://pabebezz.github.io/article/2d45afe4/  领域自适应方法
        #http://www.atyun.com/18356.html 对抗网络
        self.cnn_optimizer.zero_grad()
        self.generate_optimizer.zero_grad()

        generate_time=int(x.shape[0]*self.generate_rate)
        x_batch=x.shape[0]
        if x.shape[1]==500:
            a=tc.zeros(generate_time,152,dtype=tc.bool,device=x.device)
        else :
            a=tc.zeros(generate_time*4,152,dtype=tc.bool,device=x.device)
            
        for cache in a:
            for i in range(np.random.randint(0,5)):
                cache[np.random.randint(0,152)]=True
        more_train=self.generate.forward(a)
        
        y=tc.cat((y,a),dim=0)
        a=tc.zeros(y.shape[0],1,dtype=tc.float32,device=x.device)
        a[x_batch:,0]=1
        y=tc.cat((y,a),dim=1)
        
        if x.shape[1]==500:
            x=tc.cat((x,more_train),dim=0)
            output=self.cnn(x,True)
        else :
            x=tc.cat((x,more_train.reshape(x.shape[0],2000,-1)),dim=0)
            output=self.forward(x,True)
        if not (mask is None):
            output=output[:,mask]
        loss_sum=[]
        loss=self.loss_fc.forward(output[x_batch:,:152],y[x_batch:,:152])/generate_time#referee
        loss.backward(retain_graph=True)
        loss_sum.append(loss)
        self.cnn_optimizer.zero_grad()#清空梯度
        #referee=0 feature=0 output=0 generate_time=dloss/dx #x=data
        #print(output[:,-1],y[:,-1])
        #input()
        loss=self.loss_fc.forward(output[:,-1],y[:,-1])/output.shape[0]
        loss_sum.append(loss)
        
        #referee=dr/dx feature=dr/dx  generate_time=dloss1/dx+dr/dx #r=referee x=data
        """
        loss.backward(retain_graph=True)
        for cache in self.cnn.feature_data():
            cache:tc.Tensor
            cache.grad*=-tc.ones(1,device=x.device)"""
        #referee=dr/dx feature=-dr/dx  generate_time=dloss1/dx+dr/dx #r=referee x=data
        self.generate_optimizer.step()
        #referee=dr/dx feature=-dr/dx  generate_time=0 #r=referee x=data
        """loss=self.loss_fc.forward(output[:x_batch,:152],y[:x_batch,:152])/x_batch#referee
        loss.backward(retain_graph=False,inputs=list(self.cnn.parameters()))
        #referee=dr/dx feature=-dr/dx+dloss1/dx generate_time=0
        loss_sum.append(loss)
        
        self.cnn_optimizer.step()
        self.cnn_optimizer.zero_grad()"""
        #self.generate_optimizer.zero_grad()
        #referee=0 feature=0 generate_time=0
        
        
        return {"output":output[:x_batch,:152],"loss":tc.stack(loss_sum)}
    
    def train_generate(self,time):
        a=tc.zeros(time,152,dtype=tc.bool,device=data["device"])
        for cache in a:
            for i in range(np.random.randint(1,5)):
                cache[np.random.randint(0,152)]=True
        self.generate_optimizer.zero_grad()
        more_train=self.generate.forward(a.to(tc.float32))
        output=self.cnn(more_train,False)#main_point

        loss=self.loss_fc.forward(output,a.to(tc.float32))/time#referee
        loss.backward()
        self.generate_optimizer.step()
        self.cnn_optimizer.zero_grad()
                
        return {"output":output,"loss":tc.tensor([loss])}

    def test(self,x:tc.Tensor):
        #x.shape=(batch,500,257)
        output=self.cnn(x)
        output=output>=0
        return output
       
    def test_error(self,x:tc.Tensor):
        if x.shape[-2]!=2000:
            print(x.shape)
            print("test_error")
            exit()
        cache=x.reshape(x.shape[0],4,500,-1)
        max_=cache.max(dim=3)[0].max(dim=2)[0][:,:,None,None]
        min_=cache.min(dim=3)[0].min(dim=2)[0][:,:,None,None]
        cache=(min_-cache)/(min_-max_)
        output=self.cnn.forward(cache.reshape(x.shape[0]*4,500,-1))
        #del cache
        output1=output.reshape(x.shape[0],4,output.shape[-1])
        output=output1.max(dim=1)[0]
        #output=tc.sigmoid(output)

        return output,output1,cache.reshape(x.shape[0],500*4,-1)

class testDataset(tc.utils.data.Dataset):
    """Some Information about MyDataset"""
    def __init__(self,paths:list,bird_index:tc.Tensor,filepath:str,end_time:list,fit_randn=0.01):
        super().__init__()
        if not isinstance(paths,np.ndarray):
            paths=np.array(paths,dtype=np.object0)
        self.filepath=filepath
        self.paths=paths
        self.end_time=end_time
        self.randn=fit_randn
        self.bird_index=bird_index

    

    def __getitem__(self, index):
        def get(index):
            return self.paths[index]
        
        path=get(index)
        path=self.filepath+path+".ogg"
        if os.path.isfile(path):
            voice_data=preprocess_voice(path)
        else: 
            cache=tc.zeros(self.bird_index.shape[1],dtype=tc.bool)
            cache[0]=1
            return tc.randn(500,120,dtype=tc.float32)*0.01,cache

        time_=int(self.end_time[index])#s
        
        if voice_data.shape[0]<time_*100:
            #print("add error")
            voice_data=voice_data[-500:]
        else :
            voice_data=voice_data[time_*100-500:time_*100]
        
        """more Data_Augmentation"""
        #voice_data.shape=(500,128)
        cache=np.random.randint(1,7)
        voice_data=voice_data[:,cache:-8+cache]#voice_data.shape=(500,120)#-(8-cache)
        #d1=voice_data.max()*0.8
        #voice_data=tc.cat((tc.rand(500,cache)*d1,voice_data,tc.rand(500,8-cache)*d1),dim=1)
        #voice_data.shape=(500,n+120+8-n)=(500,120)
        #randn=tc.randint(1,1000,[1])/1000*self.randn
        
        voice_data=(voice_data-voice_data.min())/(voice_data.max()-voice_data.min())
        d1=tc.var(voice_data)
        cache=tc.randn(voice_data.shape)*d1**0.5*self.randn#0.01
        voice_data+=cache#0.01
        voice_data=(voice_data-voice_data.min())/(voice_data.max()-voice_data.min())
        return voice_data.to(tc.float32),self.bird_index[index].to(tc.bool)
    
    def __len__(self):
        return len(self.paths)

########################################################
if __name__ =="__main__":
    tc.backends.cudnn.benchmark = True###new

    data={"batch":64,"skip":2,"device":"cuda:0","num_workers":2}
    data["device"] = "cuda:0" if tc.cuda.is_available() else "cpu"

    data_path=["../input/data123/","../input/data123/sAs500_loss4p0_i124_4p8_p70_2p0_p82.pth","../input/birdclef-2022/"]
    #data_path=["D:/python_data/voice/birdclef_voice/voice_data/","model_data/main.pth","D:/python_data/voice/birdclef_voice/"]


    dc=my_dic() 
    dc.load(data_path[0])
    #print(dc.data)
    with open(data_path[2]+"scored_birds.json","r")as file:
        target=ujson.load(file)
    target=dc.transform(target).sum(0).to(tc.bool)
    dc.data=np.array(dc.data)[target]
    #print(dc.data)
    #exit()
    data["scored"]=target

    model = Model()
    model.load_checkpoint(data_path[1])
    model=model.to(data["device"]).train(False)

    data_path=data_path[2]

    data["i"]=tc.arange(124,device=data["device"])[None]+tc.arange(4,device=data["device"])[:,None]
    def run(x,index):
        #x.shape=(batch,500,128)
        batch=x.shape[0]
        x=x[...,data["i"]].transpose(1,2).reshape(batch*4,500,124)
        #if x.shape[-1]==128:
        #    x=x[...,2:-2]
        with tc.no_grad():
            sigmoid_O=model.cnn.forward(x.to(data["device"]))[:,data["scored"]]
            #sigmoid_O=sigmoid_O>0.135
        sigmoid_O=sigmoid_O.reshape(batch,4,-1).mean(dim=1)
        return sigmoid_O.cpu()#[index.to(tc.bool)]
    """def run(x,index):
        if x.shape[-1]==128:
            x=x[...,2:-2]
        with tc.no_grad():
            sigmoid_O=model.cnn.forward(x.to(data["device"]))[:,data["scored"]]
        return sigmoid_O.cpu()#[index.to(tc.bool)]"""
    ##############################################################################################################################
    """with tc.no_grad():
        a=model.cnn.forward(tc.randn((data["batch"],500,128),device=data["device"]))
        del a"""

    ##############################################################################################################################

    test=pd.read_csv(data_path+"test.csv",encoding="UTF-8")

    voice_error=0

    #in test error
    start=time.time()
    target=[]
    row_id=test["row_id"]
    file_id=test["file_id"]
    end_time=test["end_time"]
    hs_voice_path=""
    voice_set=[]
    bird=test["bird"]
    bird_index=[]
    for x in range(len(file_id)):
        if hs_voice_path!=file_id[x]:
            #print(file_id[x])
            if os.path.isfile(data_path+"test_soundscapes/"+file_id[x]+".ogg"):
                voices=preprocess_voice(data_path+"test_soundscapes/"+file_id[x]+".ogg").to(tc.float32)
                voice_error=0
            else:
                voice_error=1

            hs_voice_path=file_id[x]

        if voice_error:
            #print("voice_error")
            voice=tc.randn(500,128,dtype=tc.float32)*0.1
        else:
            time_=int(end_time[x])#s
            if voices.shape[0]<time_*100:
                #print("add error")
                voice=voices[-500:]
            else :
                voice=voices[time_*100-500:time_*100]

        voice=(voice-voice.min())/(voice.max()-voice.min())
        d1=tc.var(voice)
        voice=voice+tc.randn(voice.shape)*d1**0.5*0.#0.01
        voice=(voice-voice.min())/(voice.max()-voice.min())
        voice_set.append(voice)

        bird_index.append(bird[x])

        if x%data["batch"]==0 and len(voice_set)>1:
            voice_set=tc.stack(voice_set)

            #bird_index=dc.transform(bird_index)
            a=run(voice_set,bird_index)
            voice_set=[]
            #bird_index=[]
            if isinstance(target,list):
                target=a
            else:
                target=tc.cat((target,a),dim=0)
    if len(voice_set)>=1:
        if len(voice_set)>1:
            voice_set=tc.stack(voice_set)
        else: 
            voice_set=voice_set[0][None]

        #bird_index=dc.transform(bird_index)
        a=run(voice_set,bird_index)
        if isinstance(target,list):
            target=a
        else:
            target=tc.cat((target,a),dim=0)
        
    end = time.time()
    print(end-start)
    
    plt.imshow(target[:10])
    plt.show()
    
    percentage=0.35#x in target#0.02 i think is good number :1/21/2......

    """with open("../input/data123/"+"score_bird_rate.json","r") as file:
        t=ujson.load(file).values()
        t=list(t)"""
    cache=target.sort(dim=0)
    
    
    for x in range(target.shape[1]):
        #n=math.ceil(t[x]*target.shape[0])
        n=math.ceil(percentage*target.shape[0])
        target[:,x]=False
        target[cache[1][-n:,x],x]=True
    
    plt.imshow(cache[0][:10])
    plt.show()
    plt.imshow(target[:10])
    plt.show()
    bird_index=dc.transform(bird_index)
    target=target[bird_index.to(tc.bool)].to(tc.bool)
    
    output=pd.DataFrame({"row_id":row_id,"target":target})
    output=output.set_index('row_id')
    output.to_csv("submission.csv")
    print("fanish")
    print(output[:3])
